# 使用MindSpore训练一个简单网络

本节通过MindSpore的API快速实现一个简单的神经网络模型

参考资料：

https://www.mindspore.cn/tutorials/zh-CN/r1.9/beginner/quick_start.html

In [1]:
import mindspore
from mindspore import nn
from mindspore import ops
from mindspore.dataset import vision, transforms
from mindspore.dataset import MnistDataset

## 处理数据集
MNIST数据集是机器学习领域中经典的数据集，由6W个训练样本和1W个测试样本组成，每个样本是28 * 28像素的灰度手写数字图片，共10类（0-9）。整个数据集约为50M。

MindSpore提供基于Pipeline的数据引擎，通过数据集（Dataset）和数据变换（Transforms）实现高效的数据预处理。在本教程中，我们使用Mnist数据集，自动下载完成后，使用mindspore.dataset提供的数据变换进行预处理。



>此处的示例代码依赖download，可使用命令pip install download安装。如本文档以Notebook运行时，完成安装后需要重启kernel才能执行后续代码。

In [3]:
# Download data from open datasets
from download import download

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/" \
      "notebook/datasets/MNIST_Data.zip"
path = download(url, "./", kind="zip", replace=True)



file_sizes: 100%|██████████████████████████| 10.8M/10.8M [00:00<00:00, 47.2MB/s]
Extracting zip file...
Successfully downloaded / unzipped to ./


数据下载完成后，获得数据集对象。

In [7]:
train_dataset = MnistDataset('MNIST_Data/train')
test_dataset = MnistDataset('MNIST_Data/test')

打印数据集中包含的数据列名，用于dataset的预处理。

In [6]:
print(train_dataset.get_col_names())

['image', 'label']


MindSpore的dataset使用数据处理流水线（Data Processing Pipeline），需指定map、batch、shuffle等操作。这里我们使用map对图像数据及标签进行变换处理，然后将处理好的数据集打包为大小为64的batch。

In [9]:
def datapipe(dataset, batch_size):
    image_transforms = [
        vision.Rescale(1.0 / 255.0, 0),
        vision.Normalize(mean=(0.1307,), std=(0.3081,)),
        vision.HWC2CHW()
    ]
    label_transform = transforms.TypeCast(mindspore.int32)

    dataset = dataset.map(image_transforms, 'image')
    dataset = dataset.map(label_transform, 'label')
    dataset = dataset.batch(batch_size)
    return dataset



# Map vision transforms and batch dataset
train_dataset = datapipe(train_dataset, 64)
test_dataset = datapipe(test_dataset, 64)

MindSpore的dataset使用数据处理流水线（Data Processing Pipeline），需指定map、batch和shuffle等操作。这里我们使用map对'image'列的图像数据进行变换处理，然后将处理好的数据集打包成大小为64的batch。

In [ ]:
# Map vision transforms and batch dataset
train_dataset = train_dataset.map(transforms, 'image').batch(batch_size)
test_dataset = test_dataset.map(transforms, 'image').batch(batch_size)

使用create_tuple_iterator或create_dict_iterator对数据集进行迭代。

In [10]:
for image, label in test_dataset.create_tuple_iterator():
    print(f"Shape of image [N, C, H, W]: {image.shape} {image.dtype}")
    print(f"Shape of label: {label.shape} {label.dtype}")
    break


for data in test_dataset.create_dict_iterator():
    print(f"Shape of image [N, C, H, W]: {data['image'].shape} {data['image'].dtype}")
    print(f"Shape of label: {data['label'].shape} {data['label'].dtype}")
    break


Shape of image [N, C, H, W]: (64, 1, 28, 28) Float32
Shape of label: (64,) Int32
Shape of image [N, C, H, W]: (64, 1, 28, 28) Float32
Shape of label: (64,) Int32


## 网络构建
mindspore.nn类是构建所有网络的基类，也是网络的基本单元。当用户需要自定义网络时，可以继承nn.Cell类，并重写__init__方法和construct方法。__init__包含所有网络层的定义，construct中包含数据（Tensor）的变换过程（即计算图的构造过程）。

In [11]:
# Define model
class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.dense_relu_sequential = nn.SequentialCell(
            nn.Dense(28*28, 512),
            nn.ReLU(),
            nn.Dense(512, 512),
            nn.ReLU(),
            nn.Dense(512, 10)
        )

    def construct(self, x):
        x = self.flatten(x)
        logits = self.dense_relu_sequential(x)
        return logits

model = Network()
print(model)


Network<
  (flatten): Flatten<>
  (dense_relu_sequential): SequentialCell<
    (0): Dense<input_channels=784, output_channels=512, has_bias=True>
    (1): ReLU<>
    (2): Dense<input_channels=512, output_channels=512, has_bias=True>
    (3): ReLU<>
    (4): Dense<input_channels=512, output_channels=10, has_bias=True>
    >
  >


## 模型训练

In [12]:
# Instantiate loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = nn.SGD(model.trainable_params(), 1e-2)

在模型训练中，一个完整的训练过程（step）需要实现以下三步：
1. 正向计算：模型预测结果（logits），并与正确标签（label）求预测损失（loss）。
2. 反向传播：利用自动微分机制，自动求模型参数（parameters）对于loss的梯度（gradients）。
3. 参数优化：将梯度更新到参数上。

MindSpore使用函数式自动微分机制，因此针对上述步骤需要实现：
1. 正向计算函数定义。
2. 通过函数变换获得梯度计算函数。
3. 训练函数定义，执行正向计算、反向传播和参数优化。

In [13]:
def train(model, dataset, loss_fn, optimizer):
    # Define forward function
    def forward_fn(data, label):
        logits = model(data)
        loss = loss_fn(logits, label)
        return loss, logits

    # Get gradient function
    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

    # Define function of one-step training
    def train_step(data, label):
        (loss, _), grads = grad_fn(data, label)
        loss = ops.depend(loss, optimizer(grads))
        return loss

    size = dataset.get_dataset_size()
    model.set_train()
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(data, label)

        if batch % 100 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")


除训练外，我们定义测试函数，用来评估模型的性能。

In [14]:
def test(model, dataset, loss_fn):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for data, label in dataset.create_tuple_iterator():
        pred = model(data)
        total += len(data)
        test_loss += loss_fn(pred, label).asnumpy()
        correct += (pred.argmax(1) == label).asnumpy().sum()
    test_loss /= num_batches
    correct /= total
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


训练过程需多次迭代数据集，一次完整的迭代称为一轮（epoch）。在每一轮，遍历训练集进行训练，结束后使用测试集进行预测。打印每一轮的loss值和预测准确率（Accuracy），可以看到loss在不断下降，Accuracy在不断提高。

In [15]:
epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model, train_dataset, loss_fn, optimizer)
    test(model, test_dataset, loss_fn)
print("Done!")


Epoch 1
-------------------------------


[WARNING] DEVICE(71064,ffff8f62d780,python):2022-11-18-20:18:33.596.797 [mindspore/ccsrc/plugin/device/ascend/hal/device/kernel_select_ascend.cc:330] FilterRaisedOrReducePrecisionMatchedKernelInfo] Operator:[Default/GatherD-op46] don't support int64, reduce precision from int64 to int32.
[WARNING] DEVICE(71064,ffff8f62d780,python):2022-11-18-20:18:43.700.233 [mindspore/ccsrc/plugin/device/ascend/hal/device/kernel_select_ascend.cc:330] FilterRaisedOrReducePrecisionMatchedKernelInfo] Operator:[Default/flatten-Flatten/GatherD-op146] don't support int64, reduce precision from int64 to int32.


loss: 2.303726  [  0/938]
loss: 2.289603  [100/938]
loss: 2.264207  [200/938]
loss: 2.194804  [300/938]
loss: 1.973380  [400/938]
loss: 1.542460  [500/938]
loss: 0.977717  [600/938]
loss: 0.889897  [700/938]
loss: 0.723557  [800/938]
loss: 0.536497  [900/938]


[WARNING] DEVICE(71064,ffff8f62d780,python):2022-11-18-20:19:07.900.180 [mindspore/ccsrc/plugin/device/ascend/hal/device/kernel_select_ascend.cc:330] FilterRaisedOrReducePrecisionMatchedKernelInfo] Operator:[Default/GatherD-op246] don't support int64, reduce precision from int64 to int32.
[WARNING] DEVICE(71064,ffff8f62d780,python):2022-11-18-20:19:16.377.458 [mindspore/ccsrc/plugin/device/ascend/hal/device/kernel_select_ascend.cc:330] FilterRaisedOrReducePrecisionMatchedKernelInfo] Operator:[Default/flatten-Flatten/GatherD-op346] don't support int64, reduce precision from int64 to int32.
[WARNING] DEVICE(71064,ffff8f62d780,python):2022-11-18-20:19:30.646.765 [mindspore/ccsrc/plugin/device/ascend/hal/device/kernel_select_ascend.cc:330] FilterRaisedOrReducePrecisionMatchedKernelInfo] Operator:[Default/GatherD-op446] don't support int64, reduce precision from int64 to int32.


Test: 
 Accuracy: 85.3%, Avg loss: 0.529036 

Epoch 2
-------------------------------


...

Test: 
 Accuracy: 90.1%, Avg loss: 0.334975 

Epoch 3
-------------------------------


[WARNING] DEVICE(71064,ffff8f62d780,python):2022-11-18-20:19:58.582.685 [mindspore/ccsrc/plugin/device/ascend/hal/device/kernel_select_ascend.cc:330] FilterRaisedOrReducePrecisionMatchedKernelInfo] Operator:[Default/flatten-Flatten/GatherD-op790] don't support int64, reduce precision from int64 to int32.


loss: 0.587514  [  0/938]
loss: 0.245308  [100/938]
loss: 0.244951  [200/938]
loss: 0.487564  [300/938]
loss: 0.264339  [400/938]
loss: 0.288197  [500/938]
loss: 0.438579  [600/938]
loss: 0.301487  [700/938]
loss: 0.232899  [800/938]
loss: 0.265004  [900/938]


[WARNING] DEVICE(71064,ffff8f62d780,python):2022-11-18-20:20:10.452.858 [mindspore/ccsrc/plugin/device/ascend/hal/device/kernel_select_ascend.cc:330] FilterRaisedOrReducePrecisionMatchedKernelInfo] Operator:[Default/flatten-Flatten/GatherD-op911] don't support int64, reduce precision from int64 to int32.


Test: 
 Accuracy: 91.8%, Avg loss: 0.283008 

Done!


##  保存模型
模型训练完成后，需要将其参数进行保存

In [16]:
# Save checkpoint
mindspore.save_checkpoint(model, "model.ckpt")
print("Saved Model to model.ckpt")

Saved Model to model.ckpt


## 加载模型
加载保存的权重分为两步：
1. 重新实例化模型对象，构造模型。
2. 加载模型参数，并将其加载至模型上。

In [17]:
# Instantiate a random initialized model
model = Network()
# Load checkpoint and load parameter to model
param_dict = mindspore.load_checkpoint("model.ckpt")
param_not_load = mindspore.load_param_into_net(model, param_dict)
print(param_not_load)


[]


>param_not_load是未被加载的参数列表，为空时代表所有参数均加载成功。

加载后的模型可以直接用于预测推理。

In [18]:
model.set_train(False)
for data, label in test_dataset:
    pred = model(data)
    predicted = pred.argmax(1)
    print(f'Predicted: "{predicted[:10]}", Actual: "{label[:10]}"')
    break


Predicted: "[3 1 6 3 9 9 4 4 3 4]", Actual: "[3 1 5 8 9 8 4 4 3 4]"
